In [1]:
spark

In [8]:
from spark.sql import functions as f

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [4]:
latin_1 = "ISO-8859-1"

cassacao = (
    spark
    .read
    .csv("gs://gchamon-igti-deng-cloud-proc-datalake/raw/motivo_cassacao_2020_BRASIL.csv",
         sep=";",
         encoding=latin_1,
         escape="\"",
         header=True)
)

In [5]:
cassacao.limit(1).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,DS_MOTIVO_CASSACAO
0,19/09/2021,07:23:11,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,PI,11533,PARNAÍBA,180001061846,Ausência de requisito de registro


In [6]:
candidatos = (
    spark
    .read
    .csv("gs://gchamon-igti-deng-cloud-proc-datalake/raw/consulta_cand_2020_BRASIL.csv",
         sep=";",
         encoding=latin_1,
         escape="\"",
         header=True)
)

In [9]:
candidatos.limit(1).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,19/09/2021,07:20:36,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SC,80470,BLUMENAU,13,VEREADOR,240000755673,77999,GISELE REGINA DA CONCEIÇÃO,GISA,#NULO#,00796071950,GISELI.CONCEICAO@SANTAISABEL.COM.BR,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,77,SOLIDARIEDADE,SOLIDARIEDADE,240000071677,PARTIDO ISOLADO,SOLIDARIEDADE,1,BRASILEIRA NATA,SC,-3,BLUMENAU,11/09/1983,37,044134640990,4,FEMININO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),01,BRANCA,394,AUXILIAR DE ESCRITÓRIO E ASSEMELHADOS,125718.79,5,SUPLENTE,N,S,-1,06003101120206240088,2,DEFERIDO,2,DEFERIDO,SIM


In [10]:
bens_candidatos = (candidatos
    spark
    .read
    .csv("gs://gchamon-igti-deng-cloud-proc-datalake/raw/bem_candidato_2020_BRASIL.csv",
         sep=";",
         encoding=latin_1,
         escape="\"",
         header=True)
)

In [27]:
bens_candidatos.limit(1).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO
0,19/09/2021,07:20:02,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,MG,51250,RODEIRO,130001231377,4,12,Casa,UMA CASA NA RUA DELTAX VIEIRA DE SOUZA 61 - BE...,"120000,00",15/11/2020,22:47:48


In [12]:
cassacao.write.mode("overwrite").parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/cassacao")
candidatos.write.mode("overwrite").parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/candidatos")
bens_candidatos.write.mode("overwrite").parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/bens_candidatos")

In [14]:
candidatos = (
    spark
    .read
    .parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/candidatos")
)
bens_candidatos = (
    spark
    .read
    .parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/bens_candidatos")
)
cassacao = (
    spark
    .read
    .parquet("gs://gchamon-igti-deng-cloud-proc-datalake/staging/cassacao")
)

In [38]:
candidatos_bens_columns = set(candidatos.columns).intersection(bens_candidatos.columns)

candidatos_bens = (
    candidatos
    .join(bens_candidatos, list(candidatos_bens_columns), "left")
)

candidatos_bens_cassacao_columns = set(candidatos_bens.columns).intersection(cassacao.columns)

candidatos_bens_cassacao = (
    candidatos_bens
    .join(cassacao, list(candidatos_bens_cassacao_columns), "left")
)

candidatos_bens_cassacao.limit(5).toPandas()

,ANO_ELEICAO,DS_ELEICAO,SG_UE,SG_UF,NM_TIPO_ELEICAO,NM_UE,DT_GERACAO,HH_GERACAO,SQ_CANDIDATO,CD_ELEICAO,CD_TIPO_ELEICAO,DT_ELEICAO,NR_TURNO,TP_ABRANGENCIA,CD_CARGO,DS_CARGO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO,DS_MOTIVO_CASSACAO
0,2020,Eleição Suplementar - Pedra Branca/CE,15113,CE,ELEIÇÃO SUPLEMENTAR,PEDRA BRANCA,19/09/2021,07:20:36,60001396006,486,1,01/08/2021,1,MUNICIPAL,11,PREFEITO,77,FRANCISCO SEVERO CARNAUBA,ROGERIO CURDULINO,#NULO#,95540784391,CASSIACE@HOTMAIL.COM,3,INAPTO,6,RENÚNCIA,COLIGAÇÃO,77,SOLIDARIEDADE,SOLIDARIEDADE,60000793265,EU ACREDITO,PDT / PT / SOLIDARIEDADE / MDB / PP / PSB / PC...,1,BRASILEIRA NATA,CE,-3,PEDRA BRANCA,29/11/1982,38,059320880752,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),01,BRANCA,601,AGRICULTOR,-1,-1,#NULO#,N,S,-1,06000456320216060059,-1,#NULO#,-1,#NULO#,NÃO,None,None,None,None,None,None,None,None
1,2020,Eleição Suplementar Antônio Carlos,40576,MG,ELEIÇÃO SUPLEMENTAR,ANTÔNIO CARLOS,19/09/2021,07:20:36,130001396849,493,1,01/08/2021,1,MUNICIPAL,12,VICE-PREFEITO,36,SEBASTIÃO DIMAS DE SALES,DIMAS SALES,#NULO#,96315750630,ELEICAO21HEBERT@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,15,MDB,MOVIMENTO DEMOCRÁTICO BRASILEIRO,130000793560,AVANÇA ANTÔNIO CARLOS,MDB / PTC / PATRIOTA,1,BRASILEIRA NATA,MG,-3,BARBACENA,20/01/1973,48,086678170256,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),03,PARDA,411,VENDEDOR DE COMÉRCIO VAREJISTA E ATACADISTA,-1,4,NÃO ELEITO,N,S,-1,06000985720216130024,2,DEFERIDO,2,DEFERIDO,SIM,None,None,None,None,None,None,None,None
2,2020,Eleição Suplementar Antônio Carlos,40576,MG,ELEIÇÃO SUPLEMENTAR,ANTÔNIO CARLOS,19/09/2021,07:20:36,130001397387,493,1,01/08/2021,1,MUNICIPAL,11,PREFEITO,25,LUIZ CARLOS DA SILVA,PRECATA,#NULO#,24985155691,ADAILTONCDEP@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,25,DEM,DEMOCRATAS,130000796059,PARTIDO ISOLADO,DEM,1,BRASILEIRA NATA,MG,-3,ANTÔNIO CARLOS,16/03/1947,74,060571430281,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),03,PARDA,999,OUTROS,-1,4,NÃO ELEITO,S,S,-1,06001071920216130024,2,DEFERIDO,2,DEFERIDO,SIM,None,None,None,None,None,None,None,None
3,2020,Eleição Suplementar Campestre,42196,MG,ELEIÇÃO SUPLEMENTAR,CAMPESTRE,19/09/2021,07:20:36,130001373023,467,1,13/06/2021,1,MUNICIPAL,12,VICE-PREFEITO,25,ZENUN ELIAS JORGE ZENUN,DR ZENUN,#NULO#,56256809653,MARQUINHOTURQUINHO@HOTMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,25,DEM,DEMOCRATAS,130000678514,CONSTRUINDO UMA NOVA HISTÓRIA,PTB / MDB / PSB / DEM,1,BRASILEIRA NATA,MG,-3,CAMPESTRE,20/07/1961,59,078676300248,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),01,BRANCA,111,MÉDICO,-1,1,ELEITO,N,S,-1,06001244320216130222,2,DEFERIDO,2,DEFERIDO,SIM,None,None,None,None,None,None,None,None
4,2020,Eleição Suplementar Espera Feliz,44830,MG,ELEIÇÃO SUPLEMENTAR,ESPERA FELIZ,19/09/2021,07:20:36,130001371804,466,1,13/06/2021,1,MUNICIPAL,11,PREFEITO,55,OZIEL GOMES DA SILVA,OZIEL GOMES,#NULO#,92238513604,PSDPARTIDO2021@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,130000674279,PARTIDO ISOLADO,PSD,1,BRASILEIRA NATA,MG,-3,ESPERA FELIZ,01/11/1973,47,13

In [37]:
candidatos.columns.duplicated()

AttributeError: 'list' object has no attribute 'duplicated'

In [32]:
candidatos_bens_cassacao.limit(5).toPandas()

,SQ_CANDIDATO,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,DS_MOTIVO_CASSACAO
0,10000644923,19/09/2021,07:20:36,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,AC,01473,TARAUACÁ,13,VEREADOR,50444,ROSANIA CONCEIÇÃO DE SOUSA,ROSSY SOUSA,#NULO#,04137062226,CELIO5050@OUTLOOK.COM,3,INAPTO,14,INDEFERIDO,PARTIDO ISOLADO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,10000057079,PARTIDO ISOLADO,PSOL,1,BRASILEIRA NATA,AC,-3,TARAUACÁ,11/03/1998,22,006974742402,4,FEMININO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),03,PARDA,999,OUTROS,37863.22,-1,#NULO#,N,S,-1,06000728620206010005,-1,#NULO#,-1,#NULO#,NÃO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,19/09/2021,07:23:11,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,AC,01473,TARAUACÁ,Ausência de requisito de registro
1,10000641356,19/09/2021,07:20:36,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,AC,01570,ASSIS BRASIL,13,VEREADOR,55000,SAMUEL SILVA DE SOUZA,SAMUEL,#NULO#,69132739249,SAMUEL.1979.0001@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,10000056306,PARTIDO ISOLADO,PSD,1,BRASILEIRA NATA,AC,-3,SENADOR GUIOMARD,21/02/1979,41,003256312461,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),03,PARDA,532,MOTORISTA DE VEÍCULOS DE TRANSPORTE DE CARGA,12307.75,5,SUPLENTE,N,S,-1,06000736820206010006,2,DEFERIDO,2,DEFERIDO,SIM,19/09/2021,07:20:02,2020,2,Eleição Ordinária,426,Eleições Municipais 2020,15/11/2020,AC,01570,ASSIS BRASIL,3,19,Outros bens imóveis,CASA DE MADEIRA,"10000,00",18/03/2021,16:26:56,None,None,None,None,None,None,None,None,None,None,None
2,40000640369,19/09/2021,07:20:36,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,AM,02038,ANORI,13,VEREADOR,15678,MARCOS ANTONIO MONTEIRO BRUNO,MARCOS BRUNO,#NULO#,81603851291,CLAUBBERC@HOTMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,15,MDB,MOVIMENTO DEMOCRÁTICO BRASILEIRO,40000056117,PARTIDO ISOLADO,MDB,1,BRASILEIRA NATA,AM,-3,MANAUS,16/05/1986,34,030879122240,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),03,PARDA,298,SERVIDOR PÚBLICO MUNICIPAL,12307.75,5,SUPLENTE,N,N,-1,06000428020206040033,2,DEFERIDO,2,DEFERIDO,SIM,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,40000708965,19/09/2021,07:20:36,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,AM,02550,MANAUS,13,VEREADOR,27112,LUIZ HENRIQUE CARIOCA,LUIZ CARIOCA,#NULO#,44520336287,VEREADORLUIZCARIOCA@GMAIL.COM,3,INAPTO,14,INDEFERIDO,PARTIDO ISOLADO,27,DC,DEMOCRACIA CRISTÃ,40000065838,PARTIDO ISOLADO,DC,1,BRASIL

In [39]:
(
    candidatos_bens_cassacao
    .write
    .mode("overwrite")
    .partitionBy("SG_UF")
    .parquet("gs://gchamon-igti-deng-cloud-proc-datalake/curated/candidatos_bens_cassacao")
)

In [ ]:
(
    candidatos_bens_cassacao
    .write
    .mode("overwrite")
    .partitionBy("SG_UF")
    .csv("gs://gchamon-igti-deng-cloud-proc-datalake/human_readable/candidatos_bens_cassacao")
)